In [66]:
# import packages
from owslib.wcs import WebCoverageService
import geopandas as gpd

In [67]:
# link to soil most probable class from SoilGrids
wcs = WebCoverageService('https://maps.isric.org/mapserv?map=/map/wrb.map',
                         version='1.0.0')

# list available layers
list(wcs.contents)

['Acrisols',
 'Albeluvisols',
 'Alisols',
 'Andosols',
 'Arenosols',
 'Calcisols',
 'Cambisols',
 'Chernozems',
 'Cryosols',
 'Durisols',
 'Ferralsols',
 'Fluvisols',
 'Gleysols',
 'Gypsisols',
 'Histosols',
 'Kastanozems',
 'Leptosols',
 'Lixisols',
 'Luvisols',
 'MostProbable',
 'Nitisols',
 'Phaeozems',
 'Planosols',
 'Plinthosols',
 'Podzols',
 'Regosols',
 'Solonchaks',
 'Solonetz',
 'Stagnosols',
 'Umbrisols',
 'Vertisols']

In [68]:
# list operations available
[op.name for op in wcs.operations]

['GetCapabilities', 'DescribeCoverage', 'GetCoverage']

In [72]:
# check coverage for one of the layers
ocs = wcs.contents['Cryosols']

# # supported CRS
print(ocs.supportedCRS)

# supported format
print(ocs.supportedFormats)

# bounding box
print(ocs.boundingboxes)

[urn:ogc:def:crs:EPSG::4326, urn:ogc:def:crs:EPSG::152160, urn:ogc:def:crs:EPSG::3857, urn:ogc:def:crs:EPSG::54009, urn:ogc:def:crs:EPSG::54012, urn:ogc:def:crs:EPSG::4326]
['GEOTIFF_BYTE']
[{'nativeSrs': 'EPSG:4326', 'bbox': (-180.0, -56.0008103940592, 179.999942400152, 83.999167206)}, {'nativeSrs': 'EPSG:4326', 'bbox': (-180.0, -56.0008103940592, 179.999942400152, 83.999167206)}]


In [70]:
# import y2y shapefile
y2y = gpd.read_file('./study_area/Y2Y_RegionBoundary_Final2013/Y2Y_RegionBoundary.shp')

# target_crs based on CRS of wcs data
target_crs = 'EPSG:4326'

# convert y2y crs
y2y = y2y.to_crs(target_crs)
print(y2y.crs)

# get bounding box of whole region
bbox = y2y.total_bounds
bbox


EPSG:4326


array([-141.03487369,   41.75172775, -108.30034623,   67.32076298])

In [71]:
# define slightly bigger bbox
bbox = (-142, 40, -107, 68)

In [45]:
# format bounding box into subset for WCS 2.0
# something didn't work the first time so going to redownload all of them with a small buffer
# just to get a slightly different output
subsets = [('X', float(bbox[0]-1), float(bbox[2]+1)), ('Y', float(bbox[1]), float(bbox[3]))]
subsets = [('X', -55, -20), ('Y', 41, 68)]
subsets

[('X', -55, -20), ('Y', 41, 68)]

In [74]:
# download single cryosols layer
# fetch map segment for layer
response = wcs.getCoverage(identifier='Cryosols',
                        crs='urn:ogc:def:crs:EPSG::4326',
                        resx=250,
                        resy=250,
                        format='GEOTIFF_BYTE')

# download data
with open('./carbon_stock_data/soiltype_soilgrids/' + 'cryosols' + '.tif', 'wb') as file:
    file.write(response.read())

In [ ]:
# # loop through layers and download each
# for layer in wcs.contents:

#     # fetch map segment for layer
#     response = wcs.getCoverage(identifier=[layer],
#                             crs='http://www.opengis.net/def/crs/EPSG/0/152160',
#                             subsets=subsets,
#                             resx=250,
#                             resy=250,
#                             format='image/tiff')
    
#     # download data
#     with open('./carbon_stock_data/soc_soilgrids/' + layer + '.tif', 'wb') as file:
#         file.write(response.read())